In [12]:
import wrds
import pandas as pd
import numpy as np
import sqlalchemy as db
import sys
from scipy.interpolate import interp1d
import os
from datetime import datetime, timedelta
from scipy.stats import norm
import math

In [13]:
# 1. Define global variables 
option_table = 'opprcd'
forward_table = 'fwdprd'
price_table = 'secprd'
ir_table = 'zerocd'
index_secid= 108105

In [14]:
wrds_user = 'eiger'
wrds_password = 'Nordwand2011'

In [15]:
# 2. Create wrds engine
conn = wrds.Connection(wrds_username=wrds_user, wrds_password=wrds_password)

Loading library list...
Done


In [16]:
# 3: Get interest rate
ir_data = conn.get_table(library='optionm', table='zerocd')
ir_data['rate'] = ir_data['rate'] / 100
ir_data['date'] = pd.to_datetime(ir_data['date'], format='%Y-%m-%d')

In [17]:
# 4: Get Index Divident Yield
div_data = conn.raw_sql(f"""select * from optionm.idxdvd where secid={index_secid}""")
div_data['rate'] = div_data['rate'] / 100
div_data['date'] = pd.to_datetime(div_data['date'], format='%Y-%m-%d')
div_data = div_data.drop(['secid'], axis=1)
div_data.rename(columns={'rate': 'div_yield'}, inplace=True)

In [18]:
# 5: Create Index  Queries
index_str = str(index_secid)

In [19]:
# here, try a different approach, where the forward data is compared to the strike price at every maturity to get ATM options
forward_df = conn.raw_sql(f"""select * from optionm.{forward_table}{2022} where secid in ({"108105"})""",date_cols=['date'])
# process forward data
forward_df['date'] = pd.to_datetime(forward_df['date'], format='%Y-%m-%d')
forward_df['expiration'] = pd.to_datetime(forward_df['expiration'], format='%Y-%m-%d')
forward_df['days_to_exp'] = (forward_df['expiration'] - forward_df['date']).dt.days
forward_df.drop(['amsettlement'], axis=1, inplace=True)
#option_df = option_df.merge(forward_df[['secid', 'days_to_exp', 'forwardprice']],how='left', left_on=['secid', 'days_to_exp'],right_on=['secid', 'days_to_exp'])
#option_df = option_df.drop(['forward_price'], axis=1)
#option_df = option_df[(~option_df.forwardprice.isna()) | (~option_df.forwardprice.isna())]

In [20]:
forward_df

,secid,date,expiration,forwardprice,days_to_exp
0,108105.0,2022-01-03,2022-01-03,4796.560100,0
1,108105.0,2022-01-03,2022-01-05,4796.370641,2
2,108105.0,2022-01-03,2022-01-07,4796.181189,4
3,108105.0,2022-01-03,2022-01-10,4795.897025,7
4,108105.0,2022-01-03,2022-01-12,4795.707592,9
...,...,...,...,...,...
12883,108105.0,2022-12-30,2024-06-21,4063.912472,539
12884,108105.0,2022-12-30,2024-12-20,4123.974594,721
12885,108105.0,2022-12-30,2025-12-19,4274.027378,1085
12886,108105.0,2022-12-30,2026-12-18,4430.703833,1449


In [21]:
df

NameError: name 'df' is not defined

In [22]:
# 6: download option data (and spot data) and process data with some cleaning
def process_data(year):
    #download optiondata
    #volume > 0, open_interest > 0, best_bid > 0, best_bid < best_offer, gamma and impl_volatility have values
    option_df = conn.raw_sql(
    f"""
    SELECT * FROM optionm.{option_table}{year} where 
    secid in ({"108105"})
    AND volume > 0
    AND open_interest > 0
    AND best_bid > 0
    AND best_bid < best_offer
    AND gamma IS NOT NULL
    AND impl_volatility IS NOT NULL"""
    ,date_cols=['date'])
    
    #download price data
    price_df = conn.raw_sql(f"""select * from optionm.{price_table}{year} where secid in ({"108105"})""",date_cols=['date'])
    
    # process price data
    price_df['date'] = pd.to_datetime(price_df['date'], format='%Y-%m-%d')
    price_df.drop(['low', 'high', 'volume', 'return', 'cfadj', 'open', 'cfret', 'shrout'], axis=1, inplace=True)
    # process option data
    option_df = option_df.drop(['symbol_flag'], axis=1)
    option_df = option_df[option_df['ss_flag'] == '0']
    option_df = option_df.drop(['ss_flag'], axis=1)
    option_df = option_df.drop(['symbol'], axis=1)
    option_df = option_df.drop(['forward_price'], axis=1) #is none
    option_df = option_df.drop(['root'], axis=1)
    option_df = option_df.drop(['suffix'], axis=1)
    option_df = option_df.drop(['expiry_indicator'], axis=1)
    option_df = option_df.drop(['contract_size'], axis=1)
    option_df['date'] = pd.to_datetime(option_df['date'], format='%Y-%m-%d')
    option_df['exdate'] = pd.to_datetime(option_df['exdate'], format='%Y-%m-%d')
    option_df['last_date'] = pd.to_datetime(option_df['last_date'], format='%Y-%m-%d')
    option_df['days_no_trading'] = (option_df['date'] - option_df['last_date']).dt.days
    option_df = option_df.drop(['last_date'], axis=1)
    option_df.loc[option_df['days_no_trading'].isna(), 'days_no_trading'] = 0
    option_df['days_to_exp'] = (option_df['exdate'] - option_df['date']).dt.days
    days = 400 #change to 400 so we also have maturities bigger than one year in the data (for interpolation 12 month maturity)
    option_df = option_df[option_df['days_to_exp'] <= days]
    # get forward data
    forward_df = conn.raw_sql(f"""select * from optionm.{forward_table}{year} where secid in ({"108105"})""",date_cols=['date'])
    # process forward data
    forward_df['date'] = pd.to_datetime(forward_df['date'], format='%Y-%m-%d')
    forward_df['expiration'] = pd.to_datetime(forward_df['expiration'], format='%Y-%m-%d')
    forward_df['days_to_exp'] = (forward_df['expiration'] - forward_df['date']).dt.days
    forward_df.drop(['amsettlement'], axis=1, inplace=True)
    option_df = option_df.merge(forward_df[['date', 'days_to_exp', 'forwardprice']],how='left', on=['date', 'days_to_exp'])
    
    # Sort both DataFrames by 'days_to_exp' and 'secid'
    #forward_df.sort_values(['date','days_to_exp',], inplace=True)
    #option_df.sort_values(['date','days_to_exp'], inplace=True)

    # Use merge_asof to merge based on the closest value in 'days_to_exp'
    #merged_df = pd.merge_asof(option_df, forward_df[['secid', 'days_to_exp', 'forwardprice']],by='secid', on='days_to_exp', direction='nearest')
    
    # add spot data to option_df
    price_df.rename(columns={'close': 'spotprice'}, inplace=True)
    # changed how to merge the data!
    option_df = option_df.merge(price_df[['date', 'spotprice']], how='left', on=['date'])
    
    option_df['mid_price'] = (option_df['best_bid']+option_df['best_offer'])/2
    below_3 = option_df[option_df['mid_price'] < 3]
    above_3 = option_df[option_df['mid_price'] >= 3]
    below_3 = below_3[below_3['best_offer'] - below_3['best_bid'] > 0.05]
    above_3 = above_3[above_3['best_offer'] - above_3['best_bid'] > 0.1]
    option_df = pd.concat([above_3, below_3])
    
    #get ATM options
    # Calculate the difference between "strike" and "spot_price"
    option_df['strike_price'] = option_df['strike_price']/1000
    option_df['difference'] = abs(option_df['strike_price'] - option_df['forwardprice'])
    #time difference; when spot price, when option price, etc.
    # Find the row with the smallest difference for each unique combination of 'date', 'exdate', and 'type'
    option_df = option_df.loc[option_df.groupby(['date', 'exdate', 'cp_flag'])['difference'].idxmin()]
    option_df = option_df.drop(['am_settlement'], axis=1)
    
    # add interest rate
    option_df['ir_rate'] = 0
    start_date = pd.Timestamp(option_df['date'].min())
    end_date = pd.Timestamp(option_df['date'].max())
    # Get matching dates bool vector
    matching_dates_bool = (start_date <= ir_data.date) & (ir_data.date <= end_date)
    # Cut index_date appropriately
    ir_data_subset = ir_data[matching_dates_bool]
    day_before_date = start_date
    while ir_data_subset.empty:
        # If we reach this point, there was no IR data for specific date.
        # We search for the first day in the past that has a IR data
        day_before_date = day_before_date - datetime.timedelta(1)
        #Get matching dates bool vector
        matching_dates_bool = (day_before_date <= ir_data.date) & (ir_data.date <= day_before_date)
        # Cut index_date appropriately
        ir_data_subset = ir_data[matching_dates_bool]
    # - Linear Interpolation -
    # PROBLEM: Some days there is no short enough rate, so you replace with min tenor rate
    min_ir_day = ir_data_subset.days.min()
    linear_interp = interp1d(ir_data_subset['days'], ir_data_subset['rate'])
    exp_days = option_df.days_to_exp.unique()
    associated_ir_fun = lambda x: linear_interp(x).item() if x>= min_ir_day else linear_interp(min_ir_day).item()
    associated_ir = [associated_ir_fun(x) for x in exp_days]
    ir_dict = dict(zip(exp_days, associated_ir))
    option_df.loc[:,'ir_rate'] = option_df.loc[:, 'days_to_exp'].apply(lambda x: ir_dict[x])

    # Drop rows with no trading last 3 days
    option_df = option_df[option_df['days_no_trading'] < 3]  #  None is filtered too
    ## Days to expiry - too low
    # Drop rows with days to expiry 1 or 0
    option_df = option_df[option_df['days_to_exp'] > 1]
    
    option_df = option_df.drop(['secid'], axis=1)
    option_df = option_df.drop(['best_bid'], axis=1)
    option_df = option_df.drop(['best_offer'], axis=1)
    option_df = option_df.drop(['volume'], axis=1)
    option_df = option_df.drop(['open_interest'], axis=1)
    #option_df = option_df.drop(['impl_volatility'], axis=1)
    option_df = option_df.drop(['theta'], axis=1)
    #option_df = option_df.drop(['delta'], axis=1)
    #option_df = option_df.drop(['gamma'], axis=1)
    #option_df = option_df.drop(['vega'], axis=1)
    #option_df = option_df.drop(['optionid'], axis=1) keep this
    option_df = option_df.drop(['cfadj'], axis=1)
    option_df = option_df.drop(['days_no_trading'], axis=1)
    option_df = option_df.drop(['difference'], axis=1)
    
    return option_df


In [24]:
df = process_data(1996)
for year in range(1997, 2024):
    df1 = process_data(year)
    df = pd.concat([df, df1])
df = df.reset_index(drop=True)
print(df)

             date     exdate cp_flag  strike_price  impl_volatility     delta  \
0      1996-01-04 1996-01-20       C         620.0         0.112168  0.459258   
1      1996-01-04 1996-01-20       P         620.0         0.128615 -0.533466   
2      1996-01-04 1996-02-17       C         620.0         0.099041  0.505065   
3      1996-01-04 1996-02-17       P         620.0         0.119911 -0.490664   
4      1996-01-04 1996-03-16       C         620.0         0.105047  0.526898   
...           ...        ...     ...           ...              ...       ...   
203492 2023-02-28 2024-01-19       P        4150.0         0.180978 -0.484303   
203493 2023-02-28 2024-02-16       C        4175.0         0.189217  0.502561   
203494 2023-02-28 2024-02-16       P        4175.0         0.181374 -0.487211   
203495 2023-02-28 2024-03-15       C        4200.0         0.187149  0.497975   
203496 2023-02-28 2024-03-15       P        4200.0         0.181343 -0.490276   

           gamma        veg

In [25]:
# add the dividend yield data to the main dataset
option_df = df.merge(div_data[['date', 'div_yield']], on=['date'], how='left')

In [26]:
# for every option (identified with the optionid), the price from the next day is downloaded and stored
# this is the intial call
option_id = str(int(option_df['optionid'][0]))
current_day = (option_df['date'][0]).strftime("%Y-%m-%d")
next_day = option_df.loc[option_df['date'] > current_day, 'date'].min()
year = next_day.year
next_day = next_day.strftime("%Y-%m-%d")
option_df1 = conn.raw_sql(
    f"""
    SELECT date, best_bid, best_offer, optionid FROM optionm.{option_table}{year} where 
    secid in ({"108105"})
    AND optionid in ({option_id})"""
    ,date_cols=['date'])
option_df1 = option_df1[option_df1['date'] == next_day]
option_df1['mid_price'] = (option_df1['best_bid']+option_df1['best_offer'])/2
option_df1 = option_df1.drop(['best_bid'], axis=1)
option_df1 = option_df1.drop(['best_offer'], axis=1)
option_df1.rename(columns={'mid_price': 'next_price'}, inplace=True)
option_df1.at[1,'date'] = current_day

In [27]:
#now loop over optionid and then merge based on optionid
for i in range(1, option_df.loc[option_df["date"] == "2023-02-28"].index[0]):
    option_id = str(int(option_df['optionid'][i]))
    current_day = (option_df['date'][i]).strftime("%Y-%m-%d")
    next_day = option_df.loc[option_df['date'] > current_day, 'date'].min()
    year = next_day.year
    next_day = next_day.strftime("%Y-%m-%d")
    df = conn.raw_sql(
    f"""
    SELECT date, best_bid, best_offer, optionid FROM optionm.{option_table}{year} where 
    secid in ({"108105"})
    AND optionid in ({option_id})"""
    ,date_cols=['date'])
    df = df[df['date'] == next_day]
    df['mid_price'] = (df['best_bid']+df['best_offer'])/2
    df = df.drop(['best_bid'], axis=1)
    df = df.drop(['best_offer'], axis=1)
    df.rename(columns={'mid_price': 'next_price'}, inplace=True)
    df = df.reset_index(drop=True)
    df.at[0,'date'] = current_day
    option_df1 = pd.concat([option_df1, df])

In [28]:
option_df1 = option_df1.reset_index(drop=True)

# Display the resulting dataFrame with the new index
print(option_df1)

             date     optionid  next_price
0      1996-01-04   10710849.0       3.375
1      1996-01-04   10959842.0       7.125
2      1996-01-04   11326861.0       7.875
3      1996-01-04   10015587.0      10.250
4      1996-01-04   10470021.0      11.625
...           ...          ...         ...
203408 2023-02-27  149817039.0     279.250
203409 2023-02-27  152137324.0     295.450
203410 2023-02-27  152137437.0     287.250
203411 2023-02-27  151480182.0     326.500
203412 2023-02-27  151480297.0     294.400

[203413 rows x 3 columns]


In [29]:
# merge this data with the main dataset
option_df = option_df.merge(option_df1[['date','optionid', 'next_price']], on=['date','optionid'], how='left')
option_df = option_df.dropna(subset=['next_price'])
print(option_df)

             date     exdate cp_flag  strike_price  impl_volatility     delta  \
0      1996-01-04 1996-01-20       C         620.0         0.112168  0.459258   
1      1996-01-04 1996-01-20       P         620.0         0.128615 -0.533466   
2      1996-01-04 1996-02-17       C         620.0         0.099041  0.505065   
3      1996-01-04 1996-02-17       P         620.0         0.119911 -0.490664   
4      1996-01-04 1996-03-16       C         620.0         0.105047  0.526898   
...           ...        ...     ...           ...              ...       ...   
203408 2023-02-27 2024-01-19       P        4125.0         0.189797 -0.461577   
203409 2023-02-27 2024-02-16       C        4125.0         0.194080  0.535224   
203410 2023-02-27 2024-02-16       P        4125.0         0.192217 -0.452143   
203411 2023-02-27 2024-03-15       C        4100.0         0.195922  0.554199   
203412 2023-02-27 2024-03-15       P        4125.0         0.193775 -0.443975   

           gamma        veg

In [30]:
# now rearrange the full dataset, such that for every date - maturity pair, I have one row
call_price = option_df[option_df['cp_flag'] == 'C'][['date','exdate','mid_price']]
put_price = option_df[option_df['cp_flag'] == 'P'][['date','exdate','mid_price']]
print(len(call_price))
print(len(put_price))

next_call_price = option_df[option_df['cp_flag'] == 'C'][['date','exdate','next_price']]
next_put_price = option_df[option_df['cp_flag'] == 'P'][['date','exdate','next_price']]
print(len(next_call_price))
print(len(next_put_price))

option_df = option_df.groupby(['date', 'exdate']).agg(
    strike_price_c=pd.NamedAgg(column='strike_price', aggfunc='first'),
    strike_price_p=pd.NamedAgg(column='strike_price', aggfunc='last'),
    days_to_exp=pd.NamedAgg(column='days_to_exp', aggfunc='first'),
    spotprice=pd.NamedAgg(column='spotprice', aggfunc='first'),
    ir_rate=pd.NamedAgg(column='ir_rate', aggfunc='first'),
    div_yield=pd.NamedAgg(column='div_yield', aggfunc='first'),
    impl_volatility_c=pd.NamedAgg(column='impl_volatility', aggfunc='first'),
    impl_volatility_p=pd.NamedAgg(column='impl_volatility', aggfunc='last'),
    delta_c=pd.NamedAgg(column='delta', aggfunc='first'),
    delta_p=pd.NamedAgg(column='delta', aggfunc='last')
    #,gamma=pd.NamedAgg(column='gamma', aggfunc='first'), #check wheter this is the same for calls/puts
    #vega=pd.NamedAgg(column='vega', aggfunc='first') #check wheter this is the same for calls/puts
).reset_index()

option_df = option_df.merge(call_price[['date', 'exdate', 'mid_price']], on=['date','exdate'], how='left')
option_df.rename(columns={'mid_price': 'call_price'}, inplace=True)
option_df = option_df.merge(put_price[['date', 'exdate', 'mid_price']], on=['date','exdate'], how='left')
option_df.rename(columns={'mid_price': 'put_price'}, inplace=True)
option_df = option_df.merge(next_call_price[['date', 'exdate', 'next_price']], on=['date','exdate'], how='left')
option_df.rename(columns={'next_price': 'next_call_price'}, inplace=True)
option_df = option_df.merge(next_put_price[['date', 'exdate', 'next_price']], on=['date','exdate'], how='left')
option_df.rename(columns={'next_price': 'next_put_price'}, inplace=True)

option_df = option_df[['date', 'exdate', 'strike_price_c','strike_price_p','call_price',
                       'put_price', 'next_call_price', 'next_put_price',
                       'days_to_exp', 'spotprice', 'ir_rate', 'div_yield',
                       'impl_volatility_c','impl_volatility_p','delta_c','delta_p']]
option_df.dropna(inplace=True)

#comments: strike_price from the call and the put are sometimes different because of the cleaning exercises above

101239
102109
101239
102109


In [31]:
option_df.loc[2345:2355]

,date,exdate,strike_price_c,strike_price_p,call_price,put_price,next_call_price,next_put_price,days_to_exp,spotprice,ir_rate,div_yield,impl_volatility_c,impl_volatility_p,delta_c,delta_p
2345,1997-07-15,1998-03-21,950.0,900.0,57.125,36.3750,63.3750,33.250,249,925.76,0.057900,0.016898,0.186825,0.199159,0.530897,-0.330892
2346,1997-07-15,1998-06-20,925.0,950.0,85.125,61.2500,91.6250,56.375,340,925.76,0.058704,0.016898,0.191840,0.193407,0.611737,-0.427792
2347,1997-07-16,1997-07-19,935.0,935.0,4.250,3.4375,0.7500,4.625,3,936.59,0.068392,0.017253,0.170742,0.216474,0.581878,-0.434361
2348,1997-07-16,1997-08-16,940.0,935.0,20.125,18.6250,18.1875,21.500,31,936.59,0.057156,0.017253,0.193321,0.198500,0.506880,-0.453500
2349,1997-07-16,1997-09-20,945.0,945.0,28.250,31.1250,27.1250,34.375,66,936.59,0.058372,0.017253,0.186432,0.193858,0.504655,-0.491347
2350,1997-07-16,1997-12-20,950.0,950.0,47.125,44.8750,45.3750,47.625,157,936.59,0.057513,0.017253,0.189099,0.192253,0.531125,-0.461340
2351,1997-07-16,1998-03-21,950.0,950.0,63.375,51.5000,62.1250,54.375,248,936.59,0.058307,0.017253,0.188787,0.191606,0.559670,-0.428830
2352,1997-07-16,1998-06-20,995.0,975.0,56.000,67.3750,54.2500,69.750,339,936.59,0.058960,0.017253,0.183959,0.188930,0.478532,-0.459580
2353,1997-07-17,1997-08-16,935.0,935.0,20.625,21.5000,13.0000,34.250,30,931.61,0.059471,0.017482,0.203243,0.206445,0.506392,-0.492027
2354,1997-07-17,1997-09-20,940.0,940.0,29.625,31.8750,20.6250,44.625,65,931.61,0.059529,0.017482,0.198361,0.200729,0.505095,-0.491582


In [25]:
#currentyl: this part is ignored! compute straddle with the deltas instead of the betas! check which one is better
# with the above data, calculate the beta for every maturity (for every row)
# Define a custom function with the respective formula from the paper
# problem is that here, all the data used (interest rate, dividend yield, etc.) needs to be absolutely correct, because the
# the construction is dependent heavily on the construction of beta!
def calculate_beta_call(row):
    return ((row['spotprice']/row['call_price'])*  norm.cdf((math.log(row["spotprice"]/row["strike_price_c"]) + (row['ir_rate'] - row['div_yield'] + row['impl_volatility_c']**2/2)*row['days_to_exp'])
                                                           / row['impl_volatility_c']*math.sqrt(row['days_to_exp'])))
# put beta not necessarily needed (paper only uses beta from the call)
def calculate_beta_put(row):
    return (row['spotprice']/row['put_price'])*  norm.cdf((math.log(row["spotprice"]/row["strike_price_p"]) + (row['ir_rate'] - row['div_yield'] + row['impl_volatility_p']**2/2)*row['days_to_exp'])
                                                           / row['impl_volatility_p']*math.sqrt(row['days_to_exp']))
# Apply the custom function to create the 'beta' column
#option_df['beta_c'] = option_df.apply(calculate_beta_call, axis=1)
option_df['beta_p'] = option_df.apply(calculate_beta_put, axis=1)

In [32]:
print(option_df)
#save this into a .csv sheet
option_df.to_csv("full_atm_data1.csv", index=False)
#compare the result for the beta to the delta provided by WRDS
#compute returns for the call and the put

             date     exdate  strike_price_c  strike_price_p  call_price  \
0      1996-01-04 1996-01-20           620.0           620.0      4.6875   
1      1996-01-04 1996-02-17           620.0           620.0      8.2500   
2      1996-01-04 1996-03-16           620.0           620.0     12.0000   
3      1996-01-04 1996-06-22           620.0           625.0     22.5000   
6      1996-01-05 1996-01-20           615.0           615.0      6.0000   
...           ...        ...             ...             ...         ...   
103302 2023-02-27 2023-12-15          4100.0          4100.0    271.9000   
103303 2023-02-27 2023-12-29          4100.0          4100.0    278.3000   
103304 2023-02-27 2024-01-19          4100.0          4125.0    297.1000   
103305 2023-02-27 2024-02-16          4125.0          4125.0    302.5500   
103306 2023-02-27 2024-03-15          4100.0          4125.0    333.7000   

        put_price  next_call_price  next_put_price  days_to_exp  spotprice  \
0        

## Stop here: this is done in Jupyter: Make_straddle_returns

In [448]:
#get straddle price
def calculate_straddle_price(row, n):
    total_delta = abs(row['delta_c']) + abs(row['delta_p'])
    w2 = abs(row['delta_c']) / total_delta  # Calculate weights so we have a delta of zero
    w1 = abs(row['delta_p']) / total_delta
    if n == 1:
        return w1*row['call_price']+w2*row['put_price']
    else:
        return w1*row['next_call_price']+w2*row['next_put_price']

In [449]:
# apply the function to every row (every date)
option_df['current_straddle_price'] = option_df.apply(lambda row: calculate_straddle_price(row, 1), axis=1)
option_df['next_straddle_price'] = option_df.apply(lambda row: calculate_straddle_price(row, 2), axis=1)

In [450]:
# Selecting specific columns and creating a new DataFrame
new_df = option_df[['date', 'days_to_exp', 'current_straddle_price', 'next_straddle_price']].copy()
new_df

,date,days_to_exp,current_straddle_price,next_straddle_price
0,1996-01-04,16,5.757318,5.109840
1,1996-01-04,44,9.137655,9.079675
2,1996-01-04,72,11.867779,12.087773
3,1996-01-04,170,19.602408,19.529090
6,1996-01-05,15,5.316093,4.830802
...,...,...,...,...
103302,2023-02-27,291,272.506327,267.009695
103303,2023-02-27,305,265.404985,261.572730
103304,2023-02-27,326,280.863293,275.551640
103305,2023-02-27,354,298.541904,293.474002


In [451]:
# now with the straddle prices, use the formula from the appendix to get the constant maturity returns
# formula has been changed such that the weights are correct (changed the weights)
# check if it is correctly interpolated, sometime it might be necessary to extrapolate..
def interpolate_straddle_price(T, df):
    unique_dates = df['date'].drop_duplicates()
    return_data = []
    for date_value in unique_dates:
        # Filter the dataframe based on days to expiration exactly equal to the specified days
        subset_df = df[df['date'] == date_value].copy()
        date_value = date_value.strftime("%Y-%m-%d")
        exact_day = subset_df[subset_df['days_to_exp'] == T]
        next_day = df.loc[df['date'] > date_value, 'date'].min()
        if not exact_day.empty:
            # If there's an exact match, calculate the return using the exact match
            return_value = (exact_day['next_straddle_price'] / exact_day['current_straddle_price']) - 1
            return_data.append({'date': next_day, f"return_{T}": float(return_value.iloc[0])})
        else:
            # For interpolation, find two nearest days to expiration
            subset_df['days_diff'] = abs(subset_df['days_to_exp'] - T)
            nearest_two = subset_df.nsmallest(2, 'days_diff').sort_values('days_to_exp')
            # Calculate the interpolated price
            S1 = nearest_two.iloc[0]
            S2 = nearest_two.iloc[1]
            price_t1 = (abs(S1['days_to_exp'] - T) / abs(S2['days_to_exp'] - S1['days_to_exp'])) * S1['next_straddle_price'] + (abs(T - S2['days_to_exp']) / abs(S2['days_to_exp'] - S1['days_to_exp'])) * S2['next_straddle_price']
            price_t = (abs(S1['days_to_exp'] - T) / abs(S2['days_to_exp'] - S1['days_to_exp'])) * S1['current_straddle_price'] + (abs(T - S2['days_to_exp']) / abs(S2['days_to_exp'] - S1['days_to_exp'])) * S2['current_straddle_price']
            return_value = price_t1/price_t - 1
            return_data.append({'date': next_day, f"return_{T}": return_value})

    return pd.DataFrame(return_data)


In [452]:
results_30 = interpolate_straddle_price(30, new_df)
results_60 = interpolate_straddle_price(60, new_df)
results_90 = interpolate_straddle_price(90, new_df)
results_180 = interpolate_straddle_price(180, new_df)
results_270 = interpolate_straddle_price(270, new_df)
results_360 = interpolate_straddle_price(360, new_df)

1996-01-04 00:00:00
1996-01-05 00:00:00
1996-01-05 00:00:00
1996-01-08 00:00:00
1996-01-08 00:00:00
1996-01-09 00:00:00
1996-01-09 00:00:00
1996-01-10 00:00:00
1996-01-10 00:00:00
1996-01-11 00:00:00
1996-01-11 00:00:00
1996-01-12 00:00:00
1996-01-12 00:00:00
1996-01-15 00:00:00
1996-01-15 00:00:00
1996-01-16 00:00:00
1996-01-16 00:00:00
1996-01-17 00:00:00
1996-01-17 00:00:00
1996-01-18 00:00:00
1996-01-18 00:00:00
1996-01-19 00:00:00
1996-01-19 00:00:00
1996-01-22 00:00:00
1996-01-22 00:00:00
1996-01-23 00:00:00
1996-01-23 00:00:00
1996-01-24 00:00:00
1996-01-24 00:00:00
1996-01-25 00:00:00
1996-01-25 00:00:00
1996-01-26 00:00:00
1996-01-26 00:00:00
1996-01-29 00:00:00
1996-01-29 00:00:00
1996-01-30 00:00:00
1996-01-30 00:00:00
1996-01-31 00:00:00
1996-01-31 00:00:00
1996-02-01 00:00:00
1996-02-01 00:00:00
1996-02-02 00:00:00
1996-02-02 00:00:00
1996-02-05 00:00:00
1996-02-05 00:00:00
1996-02-06 00:00:00
1996-02-06 00:00:00
1996-02-07 00:00:00
1996-02-07 00:00:00
1996-02-08 00:00:00


In [453]:
merged_results = results_30.merge(results_60, on='date')
merged_results = merged_results.merge(results_90, on='date')
merged_results = merged_results.merge(results_180, on='date')
merged_results = merged_results.merge(results_270, on='date')
merged_results = merged_results.merge(results_360, on='date')
merged_results

,date,return_30,return_60,return_90,return_180,return_270,return_360
0,1996-01-05,-0.047362,0.005933,0.002039,0.015582,0.008405,0.006920
1,1996-01-08,-0.054206,-0.025427,-0.026459,-0.019554,-0.008769,-0.013060
2,1996-01-09,0.462928,0.200836,0.197305,0.094419,0.037223,0.050676
3,1996-01-10,0.122686,0.083203,0.075302,0.008955,0.014869,0.012539
4,1996-01-11,-0.149936,-0.071963,-0.071734,-0.034765,-0.022992,-0.030964
...,...,...,...,...,...,...,...
6826,2023-02-22,-0.046649,-0.035337,-0.028813,-0.021796,-0.014255,-0.012686
6827,2023-02-23,-0.043014,-0.033473,-0.027998,-0.022898,-0.017985,-0.016541
6828,2023-02-24,-0.000559,0.006665,0.009079,0.007553,0.004692,0.002961
6829,2023-02-27,-0.052030,-0.041104,-0.030785,-0.020353,-0.012987,-0.009571


In [454]:
merged_results.to_csv("straddle_returns.csv", index=False)

In [19]:
print(option_df)

             date     exdate  strike_price_c  strike_price_p  call_price  \
0      1996-01-04 1996-01-20           620.0           620.0      4.6875   
1      1996-01-04 1996-02-17           620.0           620.0      8.2500   
2      1996-01-04 1996-03-16           620.0           620.0     12.0000   
3      1996-01-04 1996-06-22           620.0           620.0     22.5000   
6      1996-01-05 1996-01-20           615.0           615.0      6.0000   
...           ...        ...             ...             ...         ...   
103302 2023-02-27 2023-12-15          3975.0          3975.0    347.7500   
103303 2023-02-27 2023-12-29          4000.0          3950.0    338.6000   
103304 2023-02-27 2024-01-19          4000.0          3975.0    357.5500   
103305 2023-02-27 2024-02-16          3975.0          3975.0    393.2500   
103306 2023-02-27 2024-03-15          4000.0          3975.0    394.0000   

        put_price  next_call_price  next_put_price  days_to_exp  spotprice  \
0        

In [20]:
option_df1 = option_df[['date','days_to_exp','call_price','put_price','next_call_price','next_put_price','spotprice','beta_c']]
print(option_df1)

             date  days_to_exp  call_price  put_price  next_call_price  \
0      1996-01-04           16      4.6875     7.0000           3.3750   
1      1996-01-04           44      8.2500    10.0000           7.8750   
2      1996-01-04           72     12.0000    11.7500          11.6250   
3      1996-01-04          170     22.5000    17.3750          22.2500   
6      1996-01-05           15      6.0000     4.8125           7.0625   
...           ...          ...         ...        ...              ...   
103302 2023-02-27          291    347.7500   223.3000         340.5000   
103303 2023-02-27          305    338.6000   218.6500         335.5000   
103304 2023-02-27          326    357.5500   231.0500         350.4500   
103305 2023-02-27          354    393.2500   239.4500         386.1500   
103306 2023-02-27          382    394.0000   247.3500         386.9500   

        next_put_price  spotprice      beta_c  
0               7.1250     617.70  131.776000  
1              

In [24]:
## continue here after having calculated the beta_c
# only keep columns in the dataset that are actually needed
#option_df = option_df[['']]
# Sort the DataFrame by 'date' and 'exdate' if it's not sorted already
unique_combinations = option_df1[['date', 'days_to_exp']]
# Create a new DataFrame to store returns
returns_df = pd.DataFrame(columns=['date', 'days_to_exp', 'r_c', 'r_p', 'r_straddle'])

# Calculate returns for each date-exdate combination
for index, row in unique_combinations.iterrows():
    current_date = row['date']
    current_days_to_exp = row['days_to_exp']

    # Filter data for the current date-exdate combination
    mask = (option_df1['date'] == current_date) & (option_df1['days_to_exp'] == current_days_to_exp)
    current_data = option_df1[mask].copy() 
    #current_data = df[(df['date'] == current_date) & (df['exdate'] == current_exdate)]

    # Calculate returns (r_c)
    current_data['r_c'] = (current_data['next_call_price'] / current_data['call_price']) - 1
    current_data['r_p'] = (current_data['next_put_price'] / current_data['put_price']) - 1
    call_part = (-current_data['call_price']*current_data['beta_c']+current_data['spotprice'])/(current_data['put_price']*current_data['beta_c'] - current_data['call_price']*current_data['beta_c']+current_data['spotprice']) * current_data['r_c']
    put_part = (current_data['put_price']*current_data['beta_c'])/(current_data['put_price']*current_data['beta_c'] - current_data['call_price']*current_data['beta_c']+current_data['spotprice']) * current_data['r_p']
    current_data['r_straddle'] = call_part + put_part
    

    # Store the results in returns_df
    returns_df = pd.concat([returns_df, current_data[['date', 'days_to_exp', 'r_straddle']]])

# Display the new DataFrame with returns
print(returns_df)


             date days_to_exp  r_c  r_p  r_straddle
0      1996-01-04          16  NaN  NaN    0.017857
1      1996-01-04          44  NaN  NaN    0.025000
2      1996-01-04          72  NaN  NaN    0.063830
3      1996-01-04         170  NaN  NaN    0.003597
6      1996-01-05          15  NaN  NaN   -0.337662
...           ...         ...  ...  ...         ...
103302 2023-02-27         291  NaN  NaN   -0.019481
103303 2023-02-27         305  NaN  NaN   -0.019666
103304 2023-02-27         326  NaN  NaN   -0.017961
103305 2023-02-27         354  NaN  NaN   -0.015870
103306 2023-02-27         382  NaN  NaN   -0.016778

[100041 rows x 5 columns]


In [28]:
returns_df = returns_df[['date', 'days_to_exp','r_straddle']]
returns_df.to_csv("straddle_returns.csv", index=False)

In [29]:
def interpolate_straddle_returns(T, df):
    unique_dates = df['date'].drop_duplicates()
    return_data = []
    for date_value in unique_dates:
        # Filter the dataframe based on days to expiration exactly equal to the specified days
        subset_df = df[df['date'] == date_value].copy()
        date_value = date_value.strftime("%Y-%m-%d")
        exact_day = subset_df[subset_df['days_to_exp'] == T]
        next_day = df.loc[df['date'] > date_value, 'date'].min()
        if not exact_day.empty:
            # If there's an exact match, calculate the return using the exact match
            return_value = exact_day['r_straddle']
            return_data.append({'date': next_day, f"return_{T}": float(return_value)})
        else:
            # For interpolation, find two nearest days to expiration
            subset_df['days_diff'] = abs(subset_df['days_to_exp'] - T)
            nearest_two = subset_df.nsmallest(2, 'days_diff').sort_values('days_to_exp')
            # Calculate the interpolated price
            S1 = nearest_two.iloc[0]
            S2 = nearest_two.iloc[1]
            return_value = (abs(S1['days_to_exp'] - T) / abs(S2['days_to_exp'] - S1['days_to_exp'])) * S1['r_straddle'] + (abs(T - S2['days_to_exp']) / abs(S2['days_to_exp'] - S1['days_to_exp'])) * S2['r_straddle']
            return_data.append({'date': next_day, f"return_{T}": return_value})

    return pd.DataFrame(return_data)

In [30]:
results_30 = interpolate_straddle_returns(30, returns_df)
results_60 = interpolate_straddle_returns(60, returns_df)
results_90 = interpolate_straddle_returns(90, returns_df)
results_180 = interpolate_straddle_returns(180, returns_df)
results_270 = interpolate_straddle_returns(270, returns_df)
results_360 = interpolate_straddle_returns(360, returns_df)

TypeError: Column 'days_diff' has dtype object, cannot use method 'nsmallest' with this dtype